In [1]:
import mne
from mne.stats import permutation_cluster_test
import os
import glob
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import make_pipeline
from copy import deepcopy as copy
plt.style.use("paper_style.mplstyle")
import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
import pingouin as pg
from cBCI.groups.performance import majority, statistics
from cBCI.plotting.accuracy import group_accuracy

DATA_FOLDER = "../data/EEG/"
TRIALS_PER_SESSION = 30
RESULT_FOLDER = "../results/"
GROUP_RESULT_FOLDER = RESULT_FOLDER
MAX_SESSIONS = 6
NUM_TR_TO_SKIP = 2
TR_PER_RUN = 235
SUBJECTS = [102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
event_id = {'TR': 1,
            'SessionStart': 2,
            'TrialStart': 3,
            'Decision': 4,
            'Confidence': 5,
            'FinalDecision': 6,
            'StimulusScreen': 7,
            'DecisionScreen': 8, 
            'ConfidenceScreen': 9,
            'FeedbackScreen': 10,
            'FinalDecisionScreen': 11,
            'TrialEnd': 12}
os.makedirs(RESULT_FOLDER, exist_ok=True)
os.makedirs(GROUP_RESULT_FOLDER, exist_ok=True)
INCORRECT_COLOR = "#ff0000"
CORRECT_COLOR = "#147ffa"

# Preprocess EEG data

In [ ]:
for SUBJECT in SUBJECTS:
    # Load EEG data corrected from MRI artifacts
    filename = glob.glob("%s%d*obs2.mff" % (DATA_FOLDER, SUBJECT))[0]
    print("Load", filename)
    raw = mne.io.read_raw_egi(filename, preload=True, exclude=[])
    # Set the montage
    renamed = {}
    for channel in raw.ch_names:
        renamed[channel] = "Cz" if channel == "E129" else channel
    raw.rename_channels(renamed)
    raw.set_montage(mne.channels.make_standard_montage("GSN-HydroCel-129"))
    # Remove any EEG reference
    raw, _ = mne.set_eeg_reference(raw, [])
    # Band-pass filter between 1 and 40 Hz
    raw.filter(1, 40, fir_design='firwin', n_jobs=8)
    # Extract events
    events = mne.find_events(raw, shortest_event=1)
    mne.write_events("%ss%d/%d_eve.fif" % (RESULT_FOLDER, SUBJECT, SUBJECT), events)
    # Order events by timestamp
    events = events[events[:, 0].argsort()]
    print("Number of TRs:", len(events[events[:, -1] == event_id['TR']]))
    # Extract epochs and subsample them
    for event, baseline, ranges in zip(["TrialStart", "StimulusScreen", "Decision", "FeedbackScreen"], 
                                       [None, (-0.6, -0.5), (-0.1, 0.0), (-0.1, 0.0)],
                                       [(-0.5, 0), (-0.6, 1.5), (-0.1, 2.0), (-0.1, 2.0)]):
        print("Extracting %s epochs" % event)
        # All epochs except for TrialStart are 2.1 seconds long, include a 100 ms baseline and are subsampled to 100 Hz
        epochs = mne.Epochs(raw, events=events, event_id=event_id[event],
                            tmin=ranges[0], tmax=ranges[1], baseline=baseline, 
                            preload=True, verbose=False)
        epochs.resample(100., npad='auto')
        all_epochs_trials = epochs.selection
        # Epochs with EEG signal bigger than 5 mV are rejected
        reject = dict(eeg=5e-3)
        epochs.drop_bad(reject=reject, verbose=False)
        is_rejected = np.array([False if i in epochs.selection else True for i in all_epochs_trials])
        np.save("%ss%d/s%d_is_rejected_%s.npy" % (RESULT_FOLDER, SUBJECT, SUBJECT, event.split("Screen")[0].lower()), is_rejected)
        print("Rejected %d epochs" % np.sum(is_rejected))
        epochs.save("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, SUBJECT, SUBJECT, event.split("Screen")[0].lower()), overwrite=True)
    if not os.path.isfile("%s../MRI/s%d/s%d_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT)):
        # Create stimulus files for fMRI analysis
        trial_index = 0
        valid_events = events[events[:, 2] < 13]
        tr_correct = []
        tr_incorrect = []
        tr_confident = []
        tr_notconfident = []
        tr_trust = []
        tr_distrust = []
        delay_between_trs = np.diff(events[events[:, 2] == event_id["TR"]][:, 0])
        for i in range(len(valid_events)):
            if valid_events[i, 2] == event_id["TR"]:
                # If the TR demarks the stimulus
                if i < (len(valid_events)-2) and (valid_events[i+1, 2] == event_id["StimulusScreen"] or
                                                 (valid_events[i+1, 2] != event_id["TR"] and 
                                                  valid_events[i+2, 2] == event_id["StimulusScreen"]) or
                                                 (valid_events[i+1, 2] != event_id["TR"] and 
                                                  valid_events[i+2, 2] != event_id["TR"] and 
                                                  valid_events[i+3, 2] == event_id["StimulusScreen"])):
                    # Trial begins
                    session_index = trial_index // TRIALS_PER_SESSION
                    trial_index += 1
                    # Get correctness from txt file
                    data_file = glob.glob("%smaps_protocol_mri_with_*-%d-1.txt" % (DATA_FOLDER, SUBJECT))[0]
                    try:
                        txt_file = np.loadtxt(data_file, dtype=str, delimiter="\n", encoding='utf-16')
                    except:
                        txt_file = np.loadtxt(data_file, dtype=str, delimiter="\n", encoding='utf-8')
                    current_trial = False
                    decision = 0
                    confidence = 0
                    agent_decision = 0
                    agent_confidence = 0
                    final_decision = 0
                    correct_answer = 0
                    for row in txt_file:
                        if "PreImageFile" in row and (SUBJECT == 101 or ("stim_%d" % (session_index+1)) in row) and ("%03d" % (trial_index - session_index*TRIALS_PER_SESSION)) in row:
                            current_trial = True
                        if "CorrectAnswer:" in row:
                            correct_answer = int(row.split(": ")[1])
                        if "Confidence" in row and "RESP" in row:
                            if len(row.split(": ")) < 2 or row.split(": ")[1] == '':
                                confidence = -1
                            else:
                                confidence = int(row.split(": ")[1])
                        if "Agent" in row and "Decision" in row:
                            if len(row.split(": ")) < 2 or row.split(": ")[1] == '?':
                                agent_decision = -1
                            else:
                                agent_decision = int(row.split(": ")[1])
                        if "Agent" in row and "Confidence" in row:
                            if len(row.split(": ")) < 2 or row.split(": ")[1] == '?':
                                agent_confidence = -1
                            else:
                                agent_confidence = int(row.split(": ")[1])
                        if "Decision" in row and "RESP" in row and "Final" in row and current_trial and correct_answer != 0:
                            if len(row.split(": ")) < 2 or row.split(": ")[1] == '':
                                final_decision = -1
                            else:
                                final_decision = int(row.split(": ")[1])
                            current_trial = False
                            break
                        if "Decision" in row and "RESP" in row and "Final" not in row and correct_answer != 0:
                            if len(row.split(": ")) < 2 or row.split(": ")[1] == '':
                                decision = -1
                            else:
                                decision = int(row.split(": ")[1])
                    tr_trust.append(0)
                    tr_distrust.append(0)
                    if current_trial == True:
                        # This is a non-valid trial as the decision has not been found, so mark it as neither correct nor incorrect
                        tr_correct.append(0)
                        tr_incorrect.append(0)
                    else:
                        if decision == correct_answer:
                            tr_correct.append(1)
                            tr_incorrect.append(0)
                        else:
                            tr_correct.append(0)
                            tr_incorrect.append(1)
                    if confidence == 3 or confidence == 4:
                        tr_confident.append(1)
                        tr_notconfident.append(0)
                    elif confidence == 1 or confidence == 2:
                        tr_confident.append(0)
                        tr_notconfident.append(1)
                    else:
                        # Confident was not reported, so exclude this trial
                        tr_confident.append(0)
                        tr_notconfident.append(0)
                # If it is the TR right before the feedback screen...
                elif i < len(valid_events)-2 and valid_events[i+1, 2] == event_id["FeedbackScreen"]:
                    tr_correct.append(0)
                    tr_incorrect.append(0)
                    tr_confident.append(0)
                    tr_notconfident.append(0)
                    if current_trial == True:
                        # This is a non-valid trial as the decision has not been found, so mark it as neither trust nor distrust
                        tr_trust.append(0)
                        tr_distrust.append(0)
                    else:
                        if (decision == agent_decision and decision == final_decision) or (decision != agent_decision and final_decision == agent_decision):
                            tr_trust.append(1)
                            tr_distrust.append(0)
                        elif (decision == agent_decision and decision != final_decision) or (decision != agent_decision and final_decision == decision):
                            tr_trust.append(0)
                            tr_distrust.append(1)
                        else:
                            tr_trust.append(0)
                            tr_distrust.append(0)
                else:
                    tr_correct.append(0)
                    tr_incorrect.append(0)
                    tr_confident.append(0)
                    tr_notconfident.append(0)
                    tr_trust.append(0)
                    tr_distrust.append(0)
        num_runs = int(round(trial_index / TRIALS_PER_SESSION, 0))
        print("num runs", num_runs)
        if len(tr_correct) > TR_PER_RUN*num_runs:
            # Remove initial TRs that are not sent to the scanner
            print("Fixing TRs")
            tr_correct = tr_correct[len(tr_correct)-TR_PER_RUN*num_runs:]
            tr_incorrect = tr_incorrect[len(tr_incorrect)-TR_PER_RUN*num_runs:]
            tr_confident = tr_confident[len(tr_confident)-TR_PER_RUN*num_runs:]
            tr_notconfident = tr_notconfident[len(tr_notconfident)-TR_PER_RUN*num_runs:]
            tr_trust = tr_trust[len(tr_trust)-TR_PER_RUN*num_runs:]
            tr_distrust = tr_distrust[len(tr_distrust)-TR_PER_RUN*num_runs:]
        if len(tr_correct) < TR_PER_RUN*num_runs:
            for i in range(TR_PER_RUN*num_runs-len(tr_correct)):
                tr_correct.append(0)
                tr_incorrect.append(0)
                tr_confident.append(0)
                tr_notconfident.append(0)
                tr_trust.append(0)
                tr_distrust.append(0)
        assert len(tr_correct) // num_runs == TR_PER_RUN
        tr_index_to_remove = []
        # Skip first NUM_TR_TO_SKIP in each run
        for run in range(num_runs):
            tr_index_to_remove += list(range(run*TR_PER_RUN, run*TR_PER_RUN+NUM_TR_TO_SKIP))
        for tr in sorted(tr_index_to_remove, reverse=True):
            del tr_correct[tr]
            del tr_incorrect[tr]
            del tr_confident[tr]
            del tr_notconfident[tr]
            del tr_trust[tr]
            del tr_distrust[tr]
        print("TOTAL VALID TRIALS", trial_index-num_runs*(NUM_TR_TO_SKIP != 0))
        print(len(tr_correct), len(tr_incorrect), len(tr_confident), len(tr_notconfident), len(tr_trust), len(tr_distrust))
        tr_file = np.vstack((tr_correct, tr_incorrect))
        os.makedirs("%s../MRI/s%d/" % (DATA_FOLDER, SUBJECT), exist_ok=True)
        np.savetxt("%s../MRI/s%d/s%d_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T, fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_correct_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 0], fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_incorrect_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 1], fmt="%d")
        tr_file = np.vstack((tr_trust, tr_distrust))
        np.savetxt("%s../MRI/s%d/s%d_stim_file_trust.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T, fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_trust_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 0], fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_distrust_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 1], fmt="%d")
        tr_file = np.vstack((tr_confident, tr_notconfident))
        np.savetxt("%s../MRI/s%d/s%d_stim_file_confidence.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T, fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_confident_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 0], fmt="%d")
        np.savetxt("%s../MRI/s%d/s%d_notconfident_stim_file.1D" % (DATA_FOLDER, SUBJECT, SUBJECT), tr_file.T[:, 1], fmt="%d")

# BCI

### Decode confidence

In [ ]:
NUM_FEATURES = 16
NUM_FOLDS = 10
behavioral_data = pd.read_csv("%s../BEHAVIOR/behavioral_data.csv" % GROUP_RESULT_FOLDER)
bci_confidence_stimulus = np.ones((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION))
bci_confidence_stimulus_predict = np.ones((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION))
bci_confidence_feedback = np.ones((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION))
bci_confidence_feedback_predict = np.ones((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION))
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)
confidence = np.ones((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION))
is_correct = np.zeros((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION), dtype=bool)
is_correct_final = np.zeros((len(SUBJECTS), MAX_SESSIONS*TRIALS_PER_SESSION), dtype=bool)
for train_index, test_index in kf.split(range(MAX_SESSIONS*TRIALS_PER_SESSION)):
    IS_TRAIN = np.zeros((MAX_SESSIONS*TRIALS_PER_SESSION), dtype=bool)
    IS_TRAIN[train_index] = 1
    for s, subject in enumerate(SUBJECTS):
        decisions = behavioral_data["Decisions_%s" % subject].values
        final_decisions = behavioral_data["Final_Decisions_%s" % subject].values
        # STIMULUS LOCKED
        epochs = mne.read_epochs("%ss%d/s%d_stimulus-epo.fif" % (RESULT_FOLDER, subject, subject), 
                                 preload=True, verbose=False)
        is_rejected = np.load("%ss%d/s%d_is_rejected_stimulus.npy" % (RESULT_FOLDER, subject, subject))
        if len(is_rejected) < MAX_SESSIONS*TRIALS_PER_SESSION:
            is_rejected = np.hstack((is_rejected, [True]))
        # Bad epochs
        bci_confidence_stimulus[s, np.where(is_rejected)] = np.nan
        # Missing responses
        bci_confidence_stimulus[s, np.where(decisions == -1)] = -1
        epochs_to_drop = np.where(decisions[~is_rejected] == -1)[0]
        bci_confidence_stimulus[s, np.where(decisions == -1)] = np.nan
        epochs.drop(epochs_to_drop)
        # This handles incomplete epochs at the end
        if len(epochs) < np.sum(~np.isnan(bci_confidence_stimulus[s])):
            bci_confidence_stimulus[s, -1] = np.nan
        # Behavioral data
        confidence[s] = behavioral_data["Confidence_%s" % subject].values
        is_correct[s] = (decisions == behavioral_data["Correct_Answer"]).values
        # Prepare epochs
        epoch_data = epochs.pick("eeg").drop_channels(["Cz"]).get_data()
        events = epochs.events
        events[is_correct[s, ~np.isnan(bci_confidence_stimulus[s])], -1] = 1
        events[~is_correct[s, ~np.isnan(bci_confidence_stimulus[s])], -1] = 2
        event_id = dict(correct=1, incorrect=2)
        raw_epochs = mne.EpochsArray(data=epoch_data, info=epochs.info, events=events, event_id=event_id)
        # Classification pipeline
        y_train = is_correct[s, ~np.isnan(bci_confidence_stimulus[s])][IS_TRAIN[~np.isnan(bci_confidence_stimulus[s])]]
        y_test = is_correct[s, ~np.isnan(bci_confidence_stimulus[s])][~IS_TRAIN[~np.isnan(bci_confidence_stimulus[s])]]
        featurizer = mne.preprocessing.Xdawn(n_components=NUM_FEATURES)
        clf = make_pipeline(featurizer, 
                            mne.decoding.Vectorizer(),
                            svm.SVC(C=1e3, probability=True, gamma='scale', class_weight="balanced", 
                                    random_state=0))
        clf.fit(raw_epochs[IS_TRAIN[~np.isnan(bci_confidence_stimulus[s])]], y_train.astype(int))
        
        train_idx = np.where(np.logical_and(IS_TRAIN, ~np.isnan(bci_confidence_stimulus[s])))
        test_idx = np.where(np.logical_and(~IS_TRAIN, ~np.isnan(bci_confidence_stimulus[s])))
        bci_confidence_stimulus[s, test_idx] = clf.predict_proba(raw_epochs[~IS_TRAIN[~np.isnan(bci_confidence_stimulus[s])]].get_data())[:, 1]
        bci_confidence_stimulus_predict[s, test_idx] = clf.predict(raw_epochs[~IS_TRAIN[~np.isnan(bci_confidence_stimulus[s])]])
        
        # FEEDBACK LOCKED
        epochs = mne.read_epochs("%ss%d/s%d_feedback-epo.fif" % (RESULT_FOLDER, subject, subject), 
                                 preload=True, verbose=False)
        is_rejected = np.load("%ss%d/s%d_is_rejected_feedback.npy" % (RESULT_FOLDER, subject, subject))
        if len(is_rejected) < MAX_SESSIONS*TRIALS_PER_SESSION:
            is_rejected = np.hstack((is_rejected, [True]))
        # Bad epochs
        bci_confidence_feedback[s, np.where(is_rejected)] = np.nan
        # Missing responses
        bci_confidence_feedback[s, np.where(final_decisions == -1)] = -1
        epochs_to_drop = np.where(final_decisions[~is_rejected] == -1)[0]
        epochs.drop(epochs_to_drop)
        bci_confidence_feedback[s, np.where(final_decisions == -1)] = np.nan
        # This handles incomplete epochs at the end
        if len(epochs) < np.sum(~np.isnan(bci_confidence_feedback[s])):
            bci_confidence_feedback[s, -1] = np.nan
        is_correct_final[s] = (final_decisions == behavioral_data["Correct_Answer"]).values
        # Prepare epochs
        epoch_data = epochs.pick("eeg").drop_channels(["Cz"]).get_data()
        events = epochs.events
        events[is_correct_final[s, ~np.isnan(bci_confidence_feedback[s])], -1] = 1
        events[~is_correct_final[s, ~np.isnan(bci_confidence_feedback[s])], -1] = 2
        event_id = dict(correct=1, incorrect=2)
        raw_epochs = mne.EpochsArray(data=epoch_data, info=epochs.info, events=events, event_id=event_id)
        # Classification pipeline
        y_train = is_correct_final[s, ~np.isnan(bci_confidence_feedback[s])][IS_TRAIN[~np.isnan(bci_confidence_feedback[s])]]
        y_test = is_correct_final[s, ~np.isnan(bci_confidence_feedback[s])][~IS_TRAIN[~np.isnan(bci_confidence_feedback[s])]]
        featurizer = mne.preprocessing.Xdawn(n_components=NUM_FEATURES)
        clf = make_pipeline(featurizer, 
                            mne.decoding.Vectorizer(),
                            svm.SVC(C=1e3, probability=True, gamma='scale', class_weight="balanced", 
                                    random_state=0))
        clf.fit(raw_epochs[IS_TRAIN[~np.isnan(bci_confidence_feedback[s])]], y_train.astype(int))
        train_idx = np.where(np.logical_and(IS_TRAIN, ~np.isnan(bci_confidence_feedback[s])))[0]
        test_idx = np.where(np.logical_and(~IS_TRAIN, ~np.isnan(bci_confidence_feedback[s])))[0]
        bci_confidence_feedback[s, test_idx] = clf.predict_proba(raw_epochs[~IS_TRAIN[~np.isnan(bci_confidence_feedback[s])]])[:, 1]
        bci_confidence_feedback_predict[s, test_idx] = clf.predict(raw_epochs[~IS_TRAIN[~np.isnan(bci_confidence_feedback[s])]])

In [ ]:
all_accuracies = []
all_accuracies_final = []
for s in range(len(SUBJECTS)):
    all_accuracies.append(accuracy_score(is_correct[s], bci_confidence_stimulus_predict[s]))
    all_accuracies_final.append(accuracy_score(bci_confidence_feedback_predict[s], is_correct_final[s]))
print(all_accuracies)
print(np.mean(all_accuracies))
print(all_accuracies_final)
print(np.mean(all_accuracies_final))

In [ ]:
print(ss.spearmanr(np.mean(is_correct, axis=1), np.nanmean(bci_confidence_stimulus, axis=1)))
print(ss.spearmanr(np.mean(is_correct_final, axis=1), np.nanmean(bci_confidence_feedback, axis=1)))

### Saving data to csv

In [ ]:
df = pd.DataFrame(bci_confidence_stimulus.T, columns=["BCIConf_%s" % s for s in SUBJECTS])
df = df.merge(pd.DataFrame(confidence.T, columns=["confidence_%s" % s for s in SUBJECTS]), left_index=True, right_index=True)
df = df.merge(pd.DataFrame(is_correct.T, columns=["is_correct_%s" % s for s in SUBJECTS]), left_index=True, right_index=True)
df = df.merge(pd.DataFrame(bci_confidence_feedback.T, columns=["BCIConf_feedback_%s" % s for s in SUBJECTS]), left_index=True, right_index=True)
df = df.merge(pd.DataFrame(is_correct_final.T, columns=["is_correct_final_%s" % s for s in SUBJECTS]), left_index=True, right_index=True)
# df = df.merge(pd.DataFrame(is_train.T, columns=["is_train"]), left_index=True, right_index=True)
df.to_csv(GROUP_RESULT_FOLDER+"bci_confidence.csv")

### Showing underconfident bias (Fig. 3B)

In [ ]:
df = pd.read_csv(GROUP_RESULT_FOLDER+"bci_confidence.csv", index_col=0)
accuracy = df[df.columns[(df.columns.to_series().str.contains('is_correct')) & (~df.columns.to_series().str.contains('final'))]].to_numpy().T.astype(int)
final_accuracy = df[df.columns[df.columns.to_series().str.contains('is_correct_final')]].to_numpy().T.astype(int)
bci_confidence = df[df.columns[(df.columns.to_series().str.contains('BCIConf')) & (~df.columns.to_series().str.contains('feedback'))]].to_numpy().T.astype(float)
confidence = df[df.columns[(df.columns.to_series().str.contains('confidence')) & (~df.columns.to_series().str.contains('hybrid'))]].to_numpy().T.astype(float)
confidence[confidence < 0] = np.nan
confidence = (confidence-1)/3
df = pd.DataFrame()
df["Confidence"] = ["Reported"]*len(confidence)*2+["BCI"]*len(bci_confidence)*2
df["Accuracy"] = ["1st decision"]*len(accuracy)+["2nd decision"]*len(final_accuracy)+["1st decision"]*len(accuracy)+["2nd decision"]*len(final_accuracy)
df["x"] = np.hstack((np.nanmean(100*accuracy, axis=1), np.nanmean(100*final_accuracy, axis=1), np.nanmean(100*accuracy, axis=1), np.nanmean(100*final_accuracy, axis=1)))
df["y"] = np.hstack((np.nanmean(100*confidence, axis=1), np.nanmean(100*confidence, axis=1), np.nanmean(100*bci_confidence, axis=1), np.nanmean(100*bci_confidence, axis=1)))
g = sns.lmplot(x="x", y="y", hue="Confidence", col="Accuracy", facet_kws={"sharey": 'row'},
               data=df, height=3, palette=dict(Reported="#ee9be4", BCI="#4a4bc7"),
               scatter_kws={"s": 25}, line_kws={"lw": 3}, seed=2, robust=False, n_boot=1000, truncate=False)
g.set(xlim=(20, 90), ylim=(20, 90), xlabel="", ylabel="", xticks=[20, 90], 
      yticks=[20, 90], title="").fig.subplots_adjust(wspace=.15)
g.axes[0, 0].plot([0, 100], [0, 100], zorder=-5, color='0.5', ls="--", lw=1)
g.axes[0, 1].plot([0, 100], [0, 100], zorder=-5, color='0.5', ls="--", lw=1)
g.axes[0, 0].grid(visible=False)
g.axes[0, 1].grid(visible=False)
print(pg.corr(np.nanmean(100*accuracy, axis=1), np.nanmean(100*confidence, axis=1), method='spearman'))
print(pg.compute_effsize(np.nanmean(100*accuracy, axis=1), np.nanmean(100*confidence, axis=1)))
print(pg.corr(np.nanmean(100*accuracy, axis=1), np.nanmean(100*bci_confidence, axis=1), method='spearman'))
print(pg.compute_effsize(np.nanmean(100*accuracy, axis=1), np.nanmean(100*bci_confidence, axis=1)))
print(pg.corr(np.nanmean(100*final_accuracy, axis=1), np.nanmean(100*confidence, axis=1), method='spearman'))
print(pg.compute_effsize(np.nanmean(100*final_accuracy, axis=1), np.nanmean(100*confidence, axis=1)))
print(pg.corr(np.nanmean(100*final_accuracy, axis=1), np.nanmean(100*bci_confidence, axis=1), method='spearman'))
print(pg.compute_effsize(np.nanmean(100*final_accuracy, axis=1), np.nanmean(100*bci_confidence, axis=1)))
plt.savefig("%saccuracy_confidence_correlations.pdf" % GROUP_RESULT_FOLDER, dpi=300)
plt.show()

## Simulate group decisions (Fig. 3A)

This plot will take a few minutes to generate as all possible groups of all sizes needs to be simulated.

In [ ]:
MAX_GROUP = None
df = pd.read_csv(GROUP_RESULT_FOLDER+"bci_confidence.csv", index_col=0)
accuracy = df[df.columns[(df.columns.to_series().str.contains('is_correct')) & (~df.columns.to_series().str.contains('final'))]].to_numpy().T.astype(int)
final_accuracy = df[df.columns[df.columns.to_series().str.contains('is_correct_final')]].to_numpy().T.astype(int)
bci_confidence = df[df.columns[(df.columns.to_series().str.contains('BCIConf')) & (~df.columns.to_series().str.contains('feedback'))]].to_numpy().T.astype(float)
hybrid_confidence = df[df.columns[(df.columns.to_series().str.contains('hybrid_confidence')) & (~df.columns.to_series().str.contains('feedback'))]].to_numpy().T.astype(float)
attention = df[df.columns[(df.columns.to_series().str.contains('attention')) & (~df.columns.to_series().str.contains('feedback'))]].to_numpy().T.astype(float)
bci_confidence_feedback = df[df.columns[df.columns.to_series().str.contains('BCIConf_feedback')]].to_numpy().T.astype(float)
hybrid_confidence_feedback = df[df.columns[df.columns.to_series().str.contains('hybrid_confidence_feedback')]].to_numpy().T.astype(float)
confidence = df[df.columns[(df.columns.to_series().str.contains('confidence')) & (~df.columns.to_series().str.contains('hybrid'))]].to_numpy().T.astype(float)
confidence[confidence < 0] = np.nan
conf_norm = (confidence-1)/3
correctness = np.ones_like(accuracy[0])
final_accuracy = df[df.columns[df.columns.to_series().str.contains('is_correct_final')]].to_numpy().T.astype(int)
bci_confidence_feedback = df[df.columns[df.columns.to_series().str.contains('BCIConf_feedback')]].to_numpy().T.astype(float)
# Majority
maj_perf = majority(accuracy.copy(), correctness.copy(), tie_policy="random_balanced", max_group_size=MAX_GROUP)
ax = group_accuracy(maj_perf, color="xkcd:grey", marker="o", show_sem=True, label="Majority 1st decision")
# Reported confidence
conf_perf = majority(accuracy.copy(), correctness.copy(), conf_norm.copy(), tie_policy="random_balanced", max_group_size=MAX_GROUP)
ax = group_accuracy(conf_perf, color="#c588c0", marker="o", show_sem=True, label="Confidence 1st decision", ax=ax)
# BCI confidence
bci_perf = majority(accuracy.copy(), correctness.copy(), bci_confidence.copy(), tie_policy="random_balanced", max_group_size=MAX_GROUP)
ax = group_accuracy(bci_perf, color="#324daa", marker="o", show_sem=True, label="BCI 1st decision", ax=ax)
# Second decision majority
correctness = np.ones_like(final_accuracy[0])
maj_final_perf = majority(final_accuracy.copy(), correctness.copy(), tie_policy="random_balanced", max_group_size=MAX_GROUP)
ax = group_accuracy(maj_final_perf, color="xkcd:grey", marker="s", ls="--", show_sem=True, label="Majority 2nd decision", ax=ax)
# Second decision BCI confidence
bci_perf_final = majority(final_accuracy.copy(), correctness.copy(), bci_confidence_feedback.copy(), tie_policy="random_balanced", max_group_size=MAX_GROUP)
ax = group_accuracy(bci_perf_final, color="#324daa", marker="s", ls='--', show_sem=True, label="BCI 2nd decision", ax=ax)
plt.legend(loc="lower right")
plt.yticks(range(60, 86, 5))
plt.xlim(0.5, 14.5)
plt.ylim(58, 85)
plt.savefig(GROUP_RESULT_FOLDER+"group_performance.pdf")
plt.show()

In [ ]:
print(statistics(maj_perf, conf_perf))
print(statistics(maj_perf, bci_perf))
print(statistics(conf_perf, bci_perf))
print(statistics(bci_perf, maj_final_perf))
print(statistics(bci_perf_final, maj_final_perf))

# Correct vs. Incorrect response analysis

In [ ]:
FRONTAL = [3, 10, 16, 18, 23, 124, 4, 11, 19, 24, 118, 5, 12, 20]
POSTERIOR = [61, 62, 78, 60, 67, 72, 77, 85, 66, 71, 76, 84, 70, 75, 83]
LEFT = [45, 46, 50, 51, 39, 40, 41, 47, 52, 42, 36, 29, 35]
FRONTALLEFT = [33, 39, 40, 34, 27, 26, 23, 24, 28, 35]
RIGHT = [108, 102, 101, 97, 115, 109, 103, 98, 92, 93, 104, 111, 110]
CENTRAL = [6, 112, 105, 87, 79, 54, 37, 30, 13, 7, 106, 80, 55, 31]
ALL = list(range(1, 129))
for out_of_scalp in [48, 119, 127, 128, 43, 49, 56, 63, 68, 73, 81, 88, 94, 99, 107, 113, 120, 125, 126, 17,
                     44, 38, 32, 114, 121, 1]:
    ALL.remove(out_of_scalp)
LOCATIONS = ["FRONTALLEFT", "FRONTAL", "POSTERIOR", "LEFT", "RIGHT", "CENTRAL", "ALL"]

In [ ]:
epochs = mne.read_epochs("%ss%d/s%d_stimulus_correct-epo.fif" % (RESULT_FOLDER, SUBJECTS[0], SUBJECTS[0]), 
                         preload=True, verbose=False)
for location in LOCATIONS:
    plt.figure(figsize=(3, 3))
    epochs.copy().pick(np.array(eval(location))-1).plot_sensors(axes=plt.gca(), show=False)
    plt.savefig("%s/electrode_location_%s.pdf" % (GROUP_RESULT_FOLDER, location.lower()))

### Create folders

In [ ]:
N_CHANNELS = 129
SR = 100 # HZ
EPOCH_LENGTH = 2.1  #s
EPOCH_TYPES = {"stimulus": [-600, 1500], "decision": [-100, 2000], "feedback": [-100, 2000]}
if not os.path.isdir("%scorrect_vs_incorrect" % GROUP_RESULT_FOLDER):
    os.makedirs("%scorrect_vs_incorrect" % GROUP_RESULT_FOLDER)
    for epoch in EPOCH_TYPES:
        os.makedirs("%scorrect_vs_incorrect/%s_locked" % (GROUP_RESULT_FOLDER, epoch))

### Split epochs in correct and incorrect trials

In [ ]:
behavioral_data = pd.read_csv("%sbehavioral_data.csv" % GROUP_RESULT_FOLDER)
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        print(subject, epoch_label)
        epochs = mne.read_epochs("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                 preload=True, verbose=False)
        is_rejected = np.load("%ss%d/s%d_is_rejected_%s.npy" % (RESULT_FOLDER, subject, subject, epoch_label))
        is_correct = (behavioral_data["Decisions_%s" % subject] == behavioral_data["Correct_Answer"]).values
        if epoch_label == "decision":
            is_correct = is_correct[behavioral_data["Decisions_%s" % subject] != -1]
        is_correct = is_correct[:is_rejected.shape[0]]
        # Remove rejected epochs
        is_correct = is_correct[~is_rejected]
        assert is_correct.shape[0] == epochs.get_data().shape[0]
        epochs[is_correct].save("%ss%d/s%d_%s_correct-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)
        epochs[~is_correct].save("%ss%d/s%d_%s_incorrect-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)

### Compute grand medians and Wilcoxon test

In [ ]:
subject_avg_correct = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
subject_avg_incorrect = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
wilcox = np.ones((len(EPOCH_TYPES), N_CHANNELS, int(EPOCH_LENGTH*SR)))
np.random.seed(2)
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        correct = mne.read_epochs("%ss%d/s%d_%s_correct-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                  preload=True, verbose=False)
        incorrect = mne.read_epochs("%ss%d/s%d_%s_incorrect-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                    preload=True, verbose=False)
        if len(correct) > len(incorrect):
            correct.drop(np.random.choice(range(len(correct)), replace=False, size=len(correct)-len(incorrect)))
        elif len(correct) < len(incorrect):
            incorrect.drop(np.random.choice(range(len(incorrect)), replace=False, size=len(incorrect)-len(correct)))
        subject_avg_correct[epoch_idx, s, :] = correct.average(method="median").data
        subject_avg_incorrect[epoch_idx, s] = incorrect.average(method="median").data
for epoch_idx in range(len(EPOCH_TYPES)):
    for chan in range(N_CHANNELS-1):
        for sample in range(subject_avg_correct.shape[-1]):
            wilcox[epoch_idx, chan-1, sample] = ss.wilcoxon(subject_avg_correct[epoch_idx, :, chan, sample], 
                                                            subject_avg_incorrect[epoch_idx, :, chan, sample])[1]

### Plot grand means

In [ ]:
for channels in ["frontalleft", "posterior"]:
    selected_channels = np.array(eval(channels.upper()))-1
    avg_correct = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    avg_incorrect = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    all_correct = None
    all_incorrect = None
    wilcox_curr = np.ones((len(EPOCH_TYPES), int(EPOCH_LENGTH*SR)))
    np.random.seed(2)
    subsample = 1
    for s, subject in enumerate(SUBJECTS):
        for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
            correct = mne.read_epochs("%ss%d/s%d_%s_correct-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                      preload=True, verbose=False)
            incorrect = mne.read_epochs("%ss%d/s%d_%s_incorrect-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                        preload=True, verbose=False)
            if all_correct is None:
                all_correct = np.median(correct._data[:, selected_channels], axis=1)
                all_incorrect = np.median(incorrect._data[:, selected_channels], axis=1)
            else:
                all_correct = np.vstack((all_correct, np.median(correct._data[:, selected_channels], axis=1)))
                all_incorrect = np.vstack((all_incorrect, np.median(incorrect._data[:, selected_channels], axis=1)))
            # Equalize number of epochs by picking random ones
            if len(correct) > len(incorrect):
                correct.drop(np.random.choice(range(len(correct)), replace=False, size=len(correct)-len(incorrect)))
            elif len(correct) < len(incorrect):
                incorrect.drop(np.random.choice(range(len(incorrect)), replace=False, size=len(incorrect)-len(correct)))
            avg_correct[epoch_idx, s] = np.mean(correct.average(method="mean").data[selected_channels], axis=0)
            avg_incorrect[epoch_idx, s] = np.mean(incorrect.average(method="mean").data[selected_channels], axis=0)
    all_correct = all_correct[:, 14:-50:subsample]
    all_incorrect = all_incorrect[:, 14:-50:subsample]
    T_obs, clusters, cluster_pv, H0 = permutation_cluster_test([all_correct, all_incorrect])
    for epoch_idx in range(len(EPOCH_TYPES)):
        for sample in range(avg_correct.shape[-1]):
            wilcox_curr[epoch_idx, sample] = ss.wilcoxon(avg_correct[epoch_idx, :, sample], 
                                                         avg_incorrect[epoch_idx, :, sample])[1]
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        fig, ax = plt.subplots(figsize=(4, 3))
        ep_start, ep_end = EPOCH_TYPES[epoch_label]
        abscissas = np.linspace(ep_start, ep_end, avg_correct.shape[-1])
        ax.axhline(0, color='gray', ls=':', lw=1)
        plt.grid(b=False)
        plt.ylim((-8, 8.25))
        plt.yticks(range(-8, 9, 8), fontsize=18)
        plt.xticks(range(ep_start+100, ep_end+1, 500), range(ep_start+100+500, ep_end+1+500, 500), fontsize=18)
        plt.xlim(ep_start+100, ep_end-500)
        p1 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_correct[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Correct", color=CORRECT_COLOR, lw=2)
        p2 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_incorrect[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Incorrect", color=INCORRECT_COLOR, lw=2)
        for i_c, c in enumerate(clusters):
            c = c[0]
            print(i_c, c, cluster_pv[i_c])
            if cluster_pv[i_c] <= 0.05:
                h = plt.axvspan(abscissas[c[0]], abscissas[c[-1]], color='yellow', alpha=0.3)
        where_is_significant = np.where(wilcox_curr[epoch_idx][14:-50:subsample] <= 0.05)[0]
        plt.scatter(abscissas[14:-50:subsample][where_is_significant], [8]*len(where_is_significant), s=40, marker=(5, 2), c='k', lw=0.6)
        plt.legend(loc="lower center", ncol=2, fontsize=14)
        plt.savefig("%scorrect_vs_incorrect/%s_locked/grand_mean_%s.pdf" % (GROUP_RESULT_FOLDER, epoch_label, channels))

In [ ]:
plt.ioff()
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    ep_start, ep_end = EPOCH_TYPES[epoch_label]
    for electrode in range(N_CHANNELS-1):
        fig, ax = plt.subplots()
        abscissas = np.linspace(ep_start, ep_end, subject_avg_correct.shape[-1])
        ax.axhline(0, color='gray', ls=':', lw=1)
        ax.axvline(0, color='gray', ls=":", lw=1)
        plt.grid(b=False)
        plt.ylim((-10, 10.2))
        plt.yticks(range(-10, 11, 5))
        plt.xticks(range(ep_start+100, ep_end+1, 250), range(ep_start+100, ep_end+1, 250))
        plt.xlim(ep_start+100, ep_end-500)
        plt.ylabel("EEG amplitude (uV)")
        plt.xlabel("Time (ms)")
        p1 = ax.plot(abscissas, 1e6*np.median(subject_avg_correct[epoch_idx, :, electrode], axis=0), label="Correct", color=CORRECT_COLOR, lw=2)
        p2 = ax.plot(abscissas, 1e6*np.median(subject_avg_incorrect[epoch_idx, :, electrode], axis=0), label="Incorrect", color=INCORRECT_COLOR, lw=2)
        where_is_significant = np.where(wilcox[epoch_idx, electrode] < 0.05)[0]
        plt.scatter(abscissas[where_is_significant], [10]*len(where_is_significant), s=5, marker="s", c='k')
        plt.legend(loc="lower left", ncol=1)
        plt.savefig("%scorrect_vs_incorrect/%s_locked/grand_avg_E%d.pdf" % (GROUP_RESULT_FOLDER, epoch_label, electrode+1))
        plt.clf()
        plt.close('all')
plt.ion()

### Plot scalp maps

In [ ]:
plt.ioff()
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    ep_start, ep_end = EPOCH_TYPES[epoch_label]
    epochs = mne.read_epochs("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, SUBJECTS[0], SUBJECTS[0], epoch_label), 
                             preload=True, verbose=False)
    for time in range(ep_start, ep_end, 10):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.5))
        abscissas = np.linspace(ep_start, ep_end, subject_avg_correct.shape[-1]+1)
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_avg_correct[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_avg_incorrect[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax2, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax2.invert_xaxis()
        ax2.invert_yaxis()
        plt.colorbar(im, ax=[ax1, ax2], ticks=[-10, 0, 10])
        ax1.set_title("Correct")
        ax2.set_title("Incorrect")
        plt.savefig("%scorrect_vs_incorrect/%s_locked/scalp_maps_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        fig, (ax1) = plt.subplots(1, 1, figsize=(2.8, 2))
        im, cn = mne.viz.plot_topomap(wilcox[epoch_idx, :, np.where(abscissas == time)[0][0]], cmap='RdBu', vmin=0.0, vmax=0.1,
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, show=False, sphere=-1,
                                      extrapolate="box", res=200)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        plt.colorbar(im, ax=[ax1], ticks=[0, 0.05, 0.1])
        plt.savefig("%scorrect_vs_incorrect/%s_locked/scalp_maps_wilcoxon_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        plt.clf()
        plt.close('all')
plt.ion()

## Frequency analysis
Is there any power difference during the stimulus presentation that predicts correctness?

Using stimulus-locked epochs, for each subject compute PSD for correct and incorrect in frontal and posterior electrodes

#### Individual PSD

In [ ]:
plt.ioff()
powers_correct = np.zeros((len(SUBJECTS), 128, 29, 210))
powers_incorrect = np.zeros((len(SUBJECTS), 128, 29, 210))
np.random.seed(2)
for s, subject in enumerate(SUBJECTS):
    print("Subject", subject)
    # Load epochs
    correct = mne.read_epochs("%ss%d/s%d_stimulus_correct-epo.fif" % (RESULT_FOLDER, subject, subject), 
                              preload=True, verbose=False)
    incorrect = mne.read_epochs("%ss%d/s%d_stimulus_incorrect-epo.fif" % (RESULT_FOLDER, subject, subject), 
                                preload=True, verbose=False)
    # Remove Cz
    correct = correct.drop_channels(["Cz"])
    incorrect = incorrect.drop_channels(["Cz"])
    # Equalize number of epochs by picking random ones
    if len(correct) > len(incorrect):
        correct.drop(np.random.choice(range(len(correct)), replace=False, size=len(correct)-len(incorrect)))
    elif len(correct) < len(incorrect):
        incorrect.drop(np.random.choice(range(len(incorrect)), replace=False, size=len(incorrect)-len(correct)))
    # Compute PSD
    freqs = np.arange(1., 30., 1.)
    power_correct = mne.time_frequency.tfr_multitaper(correct, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                      return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_correct[s] = power_correct.data
    freqs = power_correct.freqs
    times = power_correct.times
    infos = power_correct.info
    nave = power_correct.nave
    power_incorrect = mne.time_frequency.tfr_multitaper(incorrect, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                        return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_incorrect[s] = power_incorrect.data
    # Plot average PSD per subject
    if not os.path.isdir("%s/s%s/correct_vs_incorrect/" % (RESULT_FOLDER, subject)):
        os.makedirs("%s/s%s/correct_vs_incorrect/" % (RESULT_FOLDER, subject))
    for location in ["FRONTALLEFT", "POSTERIOR"]:
        power_correct.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Correct", combine='mean',
                           vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
        plt.savefig("%s/s%s/correct_vs_incorrect/psd_correct_%s.pdf" % (RESULT_FOLDER, subject, location.lower()))
        power_incorrect.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Incorrect", combine='mean',
                             vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
        plt.savefig("%s/s%s/correct_vs_incorrect/psd_incorrect_%s.pdf" % (RESULT_FOLDER, subject, location.lower()))
        plt.clf()
        plt.close('all')
plt.ion()

#### Group PSD

In [ ]:
avg_correct = mne.time_frequency.AverageTFR(info=infos, data=powers_correct.mean(axis=0),
                                            times=times, freqs=freqs, nave=nave)
avg_incorrect = mne.time_frequency.AverageTFR(info=infos, data=powers_incorrect.mean(axis=0),
                                              times=times, freqs=freqs, nave=nave)
plt.ioff()
for location in LOCATIONS:
    avg_correct.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Correct", combine='mean',
                     vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
    plt.savefig("%s/correct_vs_incorrect/stimulus_locked/psd_correct_%s.pdf" % (GROUP_RESULT_FOLDER, location.lower()))
    avg_incorrect.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Incorrect", combine='mean',
                       vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
    plt.savefig("%s/correct_vs_incorrect/stimulus_locked/psd_incorrect_%s.pdf" % (GROUP_RESULT_FOLDER, location.lower()))
    plt.clf()
    plt.close('all')
plt.ion()

Extract alpha, beta, etc. power for correct and incorrect

In [ ]:
bands = [['delta', 1, 4], ['theta', 4, 8], ['alpha', 8, 13], ['beta', 13, 30]]
means_per_subject = np.zeros((len(SUBJECTS), len(bands), len(LOCATIONS), 2))
time_from = 0.7
time_to = 1.4
for l, location in enumerate(LOCATIONS):
    if location != "FRONTALLEFT" and location != "POSTERIOR":
        continue
    for b, band in enumerate(bands):
        for s in range(len(SUBJECTS)):
            subj_correct = np.median(powers_correct[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 1] = 100*np.median(np.sum(subj_correct[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_correct, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
            subj_incorrect = np.median(powers_incorrect[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 0] = 100*np.median(np.sum(subj_incorrect[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_incorrect, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
        p = pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1])['p-val'].values[0]
        df = pd.DataFrame()
        df["power"] = means_per_subject[:, b, l].flatten()
        df["condition"] = ["Incorrect", "Correct"]*len(SUBJECTS)
        plt.figure(figsize=(2, 3))
        sns.boxplot(data=df, x='condition', y='power', ax=plt.gca(), notch=False, width=0.9, linewidth=1.5,
                    palette=[INCORRECT_COLOR, CORRECT_COLOR])
        sns.swarmplot(data=df, x='condition', y='power', ax=plt.gca(), color='0.2')
        plt.xlabel("")
        if band[0] == "alpha":
            plt.yticks([15, 25, 35], fontsize=18)
            plt.ylim(15, 35)
        elif band[0] == "beta":
            plt.yticks([0, 25, 50], fontsize=18)
            plt.ylim(0, 50)
        elif band[0] == "delta":
            plt.yticks([10, 35, 60], fontsize=18)
            plt.ylim(10, 60)
        elif band[0] == "theta":
            plt.yticks([20, 35, 50], fontsize=18)
            plt.ylim(20, 50)
        plt.xticks([])
        plt.ylabel("")
        plt.tight_layout()
        plt.savefig("%s/correct_vs_incorrect/eeg_%s_%s.png" % (GROUP_RESULT_FOLDER, band[0], location.lower()))
        if p > 0.0125:
            plt.clf()
            plt.close()
        else:
            print(location, band)
            print(pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            print(pg.compute_effsize(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            plt.show()

Time-frequency, p-value maps for correct vs. incorrect

In [ ]:
pvals = np.zeros((len(LOCATIONS), 29, 210))
for f in range(29):
    for t in range(210):
        for l, location in enumerate(LOCATIONS):
            pvals[l, f, t] = ss.wilcoxon(np.median(powers_correct[:, np.array(eval(location))-1, f, t], axis=1),
                                         np.median(powers_incorrect[:, np.array(eval(location))-1, f, t], axis=1))[1]

In [ ]:
all_rts = np.loadtxt("%sGROUP_RESULTS/BEHAVIOR/all_rts.txt" % RESULT_FOLDER)
all_correct_answers = np.loadtxt("%sGROUP_RESULTS/BEHAVIOR/all_correct_answers.txt" % RESULT_FOLDER)
all_decisions = np.loadtxt("%sGROUP_RESULTS/BEHAVIOR/all_decisions.txt" % RESULT_FOLDER)
mean_rt_correct = np.mean(all_rts[np.logical_and(all_decisions == all_correct_answers, all_rts > 0)])
mean_rt_incorrect = np.mean(all_rts[np.logical_and(all_decisions != all_correct_answers, all_rts > 0)])
for l, location in enumerate(LOCATIONS):
    plt.figure(figsize=(8, 8))
    plt.imshow(pvals[l], cmap="RdBu", vmin=0.0, vmax=0.1, interpolation=None, alpha=1, resample=False)
    plt.xticks(range(10, 211, 25), range(0, 2001, 250), fontsize=18)
    plt.xlim(10, 160)
    plt.yticks(range(0, 31, 10), fontsize=18)
    plt.grid(b=False)
    plt.savefig("%s/correct_vs_incorrect/stimulus_locked/psd_wilcoxon_%s.png" % (GROUP_RESULT_FOLDER, location.lower()))

In [ ]:
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    all_correct = []
    all_incorrect = []
    for s, subject in enumerate(SUBJECTS):
        correct = mne.read_epochs("%ss%d/s%d_%s_correct-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                  preload=True, verbose=False)
        all_correct.append(correct)
        incorrect = mne.read_epochs("%ss%d/s%d_%s_incorrect-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                    preload=True, verbose=False)
        all_incorrect.append(incorrect)
    correct = mne.epochs.concatenate_epochs(all_correct)
    incorrect = mne.epochs.concatenate_epochs(all_incorrect)
    correct.save("%scorrect_vs_incorrect/%s_locked/correct-epo.fif" % (GROUP_RESULT_FOLDER, epoch_label), overwrite=True)
    incorrect.save("%scorrect_vs_incorrect/%s_locked/incorrect-epo.fif" % (GROUP_RESULT_FOLDER, epoch_label), overwrite=True)

# Confidence analysis

In [ ]:
N_CHANNELS = 129
SR = 100 # HZ
EPOCH_LENGTH = 2.1  #s
EPOCH_TYPES = {"stimulus": [-600, 1500], "decision": [-100, 2000], "feedback": [-100, 2000]}
if not os.path.isdir("%sconfident_vs_notconfident" % GROUP_RESULT_FOLDER):
    os.makedirs("%sconfident_vs_notconfident" % GROUP_RESULT_FOLDER)
    for epoch in EPOCH_TYPES:
        os.makedirs("%sconfident_vs_notconfident/%s_locked" % (GROUP_RESULT_FOLDER, epoch))
CONFIDENT_COLOR = "#4dac26"
NOTCONFIDENT_COLOR = "#d01c8b"

In [ ]:
behavioral_data = pd.read_csv("%sbehavioral_data.csv" % GROUP_RESULT_FOLDER)
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        print(subject, epoch_label)
        epochs = mne.read_epochs("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                 preload=True, verbose=False)
        is_rejected = np.load("%ss%d/s%d_is_rejected_%s.npy" % (RESULT_FOLDER, subject, subject, epoch_label))
        is_notconfident = ((behavioral_data["Confidence_%s" % subject] == 1) | (behavioral_data["Confidence_%s" % subject] == 2)).values
        is_confident = ((behavioral_data["Confidence_%s" % subject] == 3) | (behavioral_data["Confidence_%s" % subject] == 4)).values
        is_confident = is_confident[:is_rejected.shape[0]]
        is_notconfident = is_notconfident[:is_rejected.shape[0]]
        # Remove rejected epochs
        is_confident = is_confident[~is_rejected]
        is_notconfident = is_notconfident[~is_rejected]
        assert is_confident.shape[0] == epochs.get_data().shape[0]
        epochs[is_confident].save("%ss%d/s%d_%s_confident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)
        epochs[is_notconfident].save("%ss%d/s%d_%s_notconfident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)

### Compute grand medians and Wilcoxon test

In [ ]:
subject_avg_confident = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
subject_avg_notconfident = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
wilcox = np.ones((len(EPOCH_TYPES), N_CHANNELS, int(EPOCH_LENGTH*SR)))
np.random.seed(2)
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        confident = mne.read_epochs("%ss%d/s%d_%s_confident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                    preload=True, verbose=False)
        notconfident = mne.read_epochs("%ss%d/s%d_%s_notconfident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                       preload=True, verbose=False)
        if len(confident) > len(notconfident):
            confident.drop(np.random.choice(range(len(confident)), replace=False, size=len(confident)-len(notconfident)))
        elif len(confident) < len(notconfident):
            notconfident.drop(np.random.choice(range(len(notconfident)), replace=False, size=len(notconfident)-len(confident)))
        subject_avg_confident[epoch_idx, s, :] = confident.average(method="median").data
        subject_avg_notconfident[epoch_idx, s] = notconfident.average(method="median").data
for epoch_idx in range(len(EPOCH_TYPES)):
    for chan in range(N_CHANNELS-1):
        for sample in range(subject_avg_confident.shape[-1]):
            wilcox[epoch_idx, chan-1, sample] = ss.wilcoxon(subject_avg_confident[epoch_idx, :, chan, sample], 
                                                            subject_avg_notconfident[epoch_idx, :, chan, sample])[1]

### Plot grand medians

In [ ]:
for channels in ["frontalleft", "posterior"]:
    selected_channels = np.array(eval(channels.upper()))-1
    avg_confident = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    avg_notconfident = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    wilcox_curr = np.ones((len(EPOCH_TYPES), int(EPOCH_LENGTH*SR)))
    all_confident = None
    all_notconfident = None
    np.random.seed(2)
    subsample = 2
    for s, subject in enumerate(SUBJECTS):
        for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
            confident = mne.read_epochs("%ss%d/s%d_%s_confident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                        preload=True, verbose=False)
            notconfident = mne.read_epochs("%ss%d/s%d_%s_notconfident-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                           preload=True, verbose=False)
            if all_confident is None:
                all_confident = np.median(confident._data[:, selected_channels], axis=1)
                all_notconfident = np.median(notconfident._data[:, selected_channels], axis=1)
            else:
                all_confident = np.vstack((all_confident, np.median(confident._data[:, selected_channels], axis=1)))
                all_notconfident = np.vstack((all_notconfident, np.median(notconfident._data[:, selected_channels], axis=1)))
            # Equalize number of epochs by picking random ones
            if len(confident) > len(notconfident):
                confident.drop(np.random.choice(range(len(confident)), replace=False, size=len(confident)-len(notconfident)))
            elif len(confident) < len(notconfident):
                notconfident.drop(np.random.choice(range(len(notconfident)), replace=False, size=len(notconfident)-len(confident)))
            avg_confident[epoch_idx, s] = np.median(confident.average(method="median").data[selected_channels], axis=0)
            avg_notconfident[epoch_idx, s] = np.median(notconfident.average(method="median").data[selected_channels], axis=0)
    all_confident = all_confident[:, 14:-50:subsample]
    all_notconfident = all_notconfident[:, 14:-50:subsample]
    T_obs, clusters, cluster_pv, H0 = permutation_cluster_test([all_confident, all_notconfident])
    for epoch_idx in range(len(EPOCH_TYPES)):
        for sample in range(avg_confident.shape[-1]):
            wilcox_curr[epoch_idx, sample] = ss.wilcoxon(avg_confident[epoch_idx, :, sample], 
                                                         avg_notconfident[epoch_idx, :, sample])[1]
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        fig, ax = plt.subplots(figsize=(4, 3))
        ep_start, ep_end = EPOCH_TYPES[epoch_label]
        abscissas = np.linspace(ep_start, ep_end, avg_confident.shape[-1])
        ax.axhline(0, color='gray', ls=':', lw=1)
        plt.grid(b=False)
        plt.ylim((-8, 8.25))
        plt.yticks(range(-8, 9, 8), fontsize=18)
        plt.xticks(range(ep_start+100, ep_end+1, 500), range(ep_start+100+500, ep_end+1+500, 500), fontsize=18)
        plt.xlim(ep_start+100, ep_end-500)
        p1 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_confident[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Confident", color=CONFIDENT_COLOR, lw=2)
        p2 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_notconfident[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Not confident", color=NOTCONFIDENT_COLOR, lw=2)
        for i_c, c in enumerate(clusters):
            c = c[0]
            print(i_c, c, cluster_pv[i_c])
            if cluster_pv[i_c] <= 0.05:
                h = plt.axvspan(abscissas[c[0]], abscissas[c[-1]], color='yellow', alpha=0.3)
        where_is_significant = np.where(wilcox_curr[epoch_idx][14:-50:subsample] <= 0.05)[0]
        plt.scatter(abscissas[14:-50:subsample][where_is_significant], [8]*len(where_is_significant), s=40, marker=(5, 2), c='k', lw=0.6)
        plt.legend(loc="lower center", ncol=2, fontsize=14)
        plt.savefig("%sconfident_vs_notconfident/%s_locked/grand_median_%s.pdf" % (GROUP_RESULT_FOLDER, epoch_label, channels))

### Plot scalp maps

In [ ]:
plt.ioff()
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    ep_start, ep_end = EPOCH_TYPES[epoch_label]
    epochs = mne.read_epochs("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, SUBJECTS[0], SUBJECTS[0], epoch_label), 
                             preload=True, verbose=False)
    for time in range(ep_start, ep_end, 10):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.5))
        abscissas = np.linspace(ep_start, ep_end, subject_avg_confident.shape[-1]+1)
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_avg_confident[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_avg_notconfident[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax2, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax2.invert_xaxis()
        ax2.invert_yaxis()
        plt.colorbar(im, ax=[ax1, ax2], ticks=[-10, 0, 10])
        ax1.set_title("Confident")
        ax2.set_title("Not confident")
        plt.savefig("%sconfident_vs_notconfident/%s_locked/scalp_maps_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        fig, (ax1) = plt.subplots(1, 1, figsize=(2.8, 2))
        im, cn = mne.viz.plot_topomap(wilcox[epoch_idx, :, np.where(abscissas == time)[0][0]], cmap='RdBu', vmin=0.0, vmax=0.1,
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, show=False, sphere=-1,
                                      extrapolate="box", res=200)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        plt.colorbar(im, ax=[ax1], ticks=[0, 0.05, 0.1])
        plt.savefig("%sconfident_vs_notconfident/%s_locked/scalp_maps_wilcoxon_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        plt.clf()
        plt.close('all')
plt.ion()

## Frequency analysis
Is there any power difference during the stimulus presentation that predicts confidence?

#### Individual PSD

In [ ]:
plt.ioff()
powers_confident = np.zeros((len(SUBJECTS), 128, 29, 210))
powers_notconfident = np.zeros((len(SUBJECTS), 128, 29, 210))
np.random.seed(2)
for s, subject in enumerate(SUBJECTS):
    print("Subject", subject)
    # Load epochs
    confident = mne.read_epochs("%ss%d/s%d_stimulus_confident-epo.fif" % (RESULT_FOLDER, subject, subject), 
                              preload=True, verbose=False)
    notconfident = mne.read_epochs("%ss%d/s%d_stimulus_notconfident-epo.fif" % (RESULT_FOLDER, subject, subject), 
                                preload=True, verbose=False)
    # Remove Cz
    confident = confident.drop_channels(["Cz"])
    notconfident = notconfident.drop_channels(["Cz"])
    # Equalize number of epochs by picking random ones
    if len(confident) > len(notconfident):
        confident.drop(np.random.choice(range(len(confident)), replace=False, size=len(confident)-len(notconfident)))
    elif len(correct) < len(incorrect):
        notconfident.drop(np.random.choice(range(len(notconfident)), replace=False, size=len(notconfident)-len(confident)))
    # Compute PSD
    freqs = np.arange(1., 30., 1.)
    power_confident = mne.time_frequency.tfr_multitaper(confident, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                      return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_confident[s] = power_confident.data
    freqs = power_confident.freqs
    times = power_confident.times
    infos = power_confident.info
    nave = power_confident.nave
    power_notconfident = mne.time_frequency.tfr_multitaper(notconfident, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                        return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_notconfident[s] = power_notconfident.data
    # Plot average PSD per subject
    if not os.path.isdir("%s/s%s/confident_vs_notconfident/" % (RESULT_FOLDER, subject)):
        os.makedirs("%s/s%s/confident_vs_notconfident/" % (RESULT_FOLDER, subject))
    for location in ["FRONTALLEFT", "POSTERIOR"]:
        power_confident.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Confident", combine='mean',
                           vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
        plt.savefig("%s/s%s/confident_vs_notconfident/psd_confident_%s.pdf" % (RESULT_FOLDER, subject, location.lower()))
        power_notconfident.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Not confident", combine='mean',
                             vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
        plt.savefig("%s/s%s/confident_vs_notconfident/psd_notconfident_%s.pdf" % (RESULT_FOLDER, subject, location.lower()))
        plt.clf()
        plt.close('all')
plt.ion()

#### Group PSD

In [ ]:
avg_confident = mne.time_frequency.AverageTFR(info=infos, data=powers_confident.mean(axis=0),
                                              times=times, freqs=freqs, nave=nave)
avg_notconfident = mne.time_frequency.AverageTFR(info=infos, data=powers_notconfident.mean(axis=0),
                                                 times=times, freqs=freqs, nave=nave)
plt.ioff()
for location in LOCATIONS:
    avg_confident.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Confident", combine='mean',
                       vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
    plt.savefig("%s/confident_vs_notconfident/stimulus_locked/psd_confident_%s.pdf" % (GROUP_RESULT_FOLDER, location.lower()))
    avg_notconfident.plot(eval(location), baseline=(-0.1, 0), mode='logratio', title="Not confident", combine='mean',
                          vmin=-0.6, vmax=0.6, tmin=-0.1, tmax=1.0, show=False, verbose=False)
    plt.savefig("%s/confident_vs_notconfident/stimulus_locked/psd_notconfident_%s.pdf" % (GROUP_RESULT_FOLDER, location.lower()))
    plt.clf()
    plt.close('all')
plt.ion()

Time-frequency, p-value maps for correct vs. incorrect

In [ ]:
pvals = np.zeros((len(LOCATIONS), 29, 210))
for f in range(29):
    for t in range(210):
        for l, location in enumerate(LOCATIONS):
            pvals[l, f, t] = ss.wilcoxon(np.median(powers_confident[:, np.array(eval(location))-1, f, t], axis=1),
                                         np.median(powers_notconfident[:, np.array(eval(location))-1, f, t], axis=1))[1]

In [ ]:
for l, location in enumerate(LOCATIONS):
    plt.figure(figsize=(8, 8))
    plt.imshow(pvals[l], cmap="RdBu", vmin=0.0, vmax=0.1, interpolation=None, alpha=1, resample=False)
    plt.xticks(range(10, 211, 25), range(0, 2001, 250), fontsize=18)
    plt.xlim(10, 160)
    plt.yticks(range(0, 31, 10), fontsize=18)
    plt.grid(b=False)
    plt.savefig("%s/confident_vs_notconfident/stimulus_locked/psd_wilcoxon_%s.png" % (GROUP_RESULT_FOLDER, location.lower()))

Alpha, beta ...

In [ ]:
bands = [['delta', 1, 4], ['theta', 4, 8], ['alpha', 8, 13], ['beta', 13, 30]]
means_per_subject = np.zeros((len(SUBJECTS), len(bands), len(LOCATIONS), 2))
time_from = 0.9
time_to = 1.45
for l, location in enumerate(LOCATIONS):
    if location != "FRONTALLEFT" and location != "POSTERIOR":
        continue
    for b, band in enumerate(bands):
        for s in range(len(SUBJECTS)):
            subj_confident = np.median(powers_confident[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 1] = 100*np.median(np.sum(subj_confident[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_confident, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
            subj_notconfident = np.median(powers_notconfident[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 0] = 100*np.median(np.sum(subj_notconfident[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_notconfident, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
        p = pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1])['p-val'].values[0]
        df = pd.DataFrame()
        df["power"] = means_per_subject[:, b, l].flatten()
        df["condition"] = ["Not confident", "Confident"]*len(SUBJECTS)
        plt.figure(figsize=(2, 3))
        sns.boxplot(data=df, x='condition', y='power', ax=plt.gca(), notch=False, width=0.9, linewidth=1.5,
                    palette=[NOTCONFIDENT_COLOR, CONFIDENT_COLOR])
        sns.swarmplot(data=df, x='condition', y='power', ax=plt.gca(), color='0.2')
        plt.xlabel("")
        if band[0] == "alpha":
            plt.yticks([15, 25, 35], fontsize=18)
            plt.ylim(15, 35)
        elif band[0] == "beta":
            plt.yticks([0, 25, 50], fontsize=18)
            plt.ylim(0, 50)
        elif band[0] == "delta":
            plt.yticks([10, 35, 60], fontsize=18)
            plt.ylim(10, 60)
        elif band[0] == "theta":
            plt.yticks([20, 35, 50], fontsize=18)
            plt.ylim(20, 50)
        plt.xticks([])
        plt.ylabel("")
        plt.tight_layout()
        plt.savefig("%s/confident_vs_notconfident/eeg_%s_%s.png" % (GROUP_RESULT_FOLDER, band[0], location.lower()))
        if p > 0.0125:
            plt.clf()
            plt.close()
        else:
            print(location, band)
            print(pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            print(pg.compute_effsize(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            plt.show()

# Feedback analysis
Group trials on the basis of change of mind and agreement between user and agent. Then, plot the ERPs during the presentation of the feedback.

Trust is measured by agreement and change of mind (no confidence involved)

In [ ]:
def compute_trust(decisions_subject, decisions_agent, final_decisions):
    return np.logical_or(np.logical_and(decisions_subject == decisions_agent, decisions_subject == final_decisions),
                         np.logical_and(decisions_subject != decisions_agent, decisions_subject != final_decisions))

### Create folders

In [ ]:
N_CHANNELS = 129
SR = 100 # HZ
EPOCH_LENGTH = 2.1  #s
EPOCH_TYPES = {"stimulus": [-600, 1500], "decision": [-100, 2000], "feedback": [-100, 2000]}
DISTRUST_COLOR = "#e66101"
TRUST_COLOR = "#5e3c99"
if not os.path.isdir("%strust_vs_distrust" % GROUP_RESULT_FOLDER):
    os.makedirs("%strust_vs_distrust" % GROUP_RESULT_FOLDER, exist_ok=True)
    for epoch in EPOCH_TYPES:
        os.makedirs("%strust_vs_distrust/%s_locked" % (GROUP_RESULT_FOLDER, epoch), exist_ok=True)

### Split epochs in correct and incorrect trials

In [ ]:
behavioral_data = pd.read_csv("%sbehavioral_data.csv" % GROUP_RESULT_FOLDER)
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        print(subject, epoch_label)
        epochs = mne.read_epochs("%ss%d/s%d_%s-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                 preload=True, verbose=False)
        is_rejected = np.load("%ss%d/s%d_is_rejected_%s.npy" % (RESULT_FOLDER, subject, subject, epoch_label))
        is_trust = compute_trust(behavioral_data["Decisions_%s" % subject], 
                                 behavioral_data["Decisions_Agent"], 
                                 behavioral_data["Final_Decisions_%s" % subject])
        # Remove rejected epochs
        is_trust = is_trust[:is_rejected.shape[0]]
        is_trust = is_trust[~is_rejected]
        assert is_trust.shape[0] == epochs.get_data().shape[0]
        epochs[is_trust].save("%ss%d/s%d_%s_trust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)
        epochs[~is_trust].save("%ss%d/s%d_%s_distrust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), overwrite=True, verbose=False)

In [ ]:
subject_averages_trust = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
subject_averages_distrust = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), N_CHANNELS, int(EPOCH_LENGTH*SR)))
wilcox_trust = np.ones((len(EPOCH_TYPES), N_CHANNELS, int(EPOCH_LENGTH*SR)))
for s, subject in enumerate(SUBJECTS):
    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        trust = mne.read_epochs("%ss%d/s%d_%s_trust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                preload=True, verbose=False)
        distrust = mne.read_epochs("%ss%d/s%d_%s_distrust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                   preload=True, verbose=False)
        subject_averages_trust[epoch_idx, s, :] = trust.average(method="median").data
        subject_averages_distrust[epoch_idx, s] = distrust.average(method="median").data
for epoch_idx in range(len(EPOCH_TYPES)):
    for chan in range(N_CHANNELS-1):
        for sample in range(subject_averages_trust.shape[-1]):
            wilcox_trust[epoch_idx, chan-1, sample] = ss.wilcoxon(subject_averages_trust[epoch_idx, :, chan, sample], 
                                                                  subject_averages_distrust[epoch_idx, :, chan, sample])[1]

### Plot grand averages

In [ ]:
plt.ioff()
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    ep_start, ep_end = EPOCH_TYPES[epoch_label]
    for electrode in range(N_CHANNELS-1):
        fig, ax = plt.subplots()
        abscissas = np.linspace(ep_start, ep_end, subject_averages_trust.shape[-1])
        ax.axhline(0, color='gray', ls=':', lw=1)
        ax.axvline(0, color='gray', ls=":", lw=1)
        plt.grid(b=False)
        plt.ylim((-15, 15.2))
        plt.yticks(range(-15, 16, 5))
        plt.xticks(range(ep_start, ep_end+1, 100), range(ep_start, ep_end+1, 100))
        plt.xlim(0, 1000)
        plt.ylabel("EEG amplitude (uV)")
        plt.xlabel("Time (ms)")
        p1 = ax.plot(abscissas, 1e6*np.median(subject_averages_trust[epoch_idx, :, electrode], axis=0), label="Trust", color=TRUST_COLOR, lw=2)
        p2 = ax.plot(abscissas, 1e6*np.median(subject_averages_distrust[epoch_idx, :, electrode], axis=0), label="Distrust", color=DISTRUST_COLOR, lw=2)
        where_is_significant = np.where(wilcox_trust[epoch_idx, electrode] < 0.05)[0]
        plt.scatter(abscissas[where_is_significant], [15]*len(where_is_significant), s=5, marker="s", c='k')
        plt.legend(loc="lower right", ncol=1)
        plt.savefig("%strust_vs_distrust/%s_locked/grand_avg_E%d.pdf" % (GROUP_RESULT_FOLDER, epoch_label, electrode+1))
        plt.clf()
        plt.close('all')
plt.ion()

In [ ]:
for channels in ['central', 'posterior']:
    selected_channels = np.array(eval(channels.upper()))-1
    avg_trust = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    avg_distrust = np.zeros((len(EPOCH_TYPES), len(SUBJECTS), int(EPOCH_LENGTH*SR)))
    wilcox_curr = np.ones((len(EPOCH_TYPES), int(EPOCH_LENGTH*SR)))
    np.random.seed(2)
    subsample = 2
    for s, subject in enumerate(SUBJECTS):
        for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
            trust = mne.read_epochs("%ss%d/s%d_%s_trust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                      preload=True, verbose=False)
            distrust = mne.read_epochs("%ss%d/s%d_%s_distrust-epo.fif" % (RESULT_FOLDER, subject, subject, epoch_label), 
                                        preload=True, verbose=False)
            # Equalize number of epochs by picking random ones
            if len(trust) > len(distrust):
                trust.drop(np.random.choice(range(len(trust)), replace=False, size=len(trust)-len(distrust)))
            elif len(trust) < len(distrust):
                distrust.drop(np.random.choice(range(len(distrust)), replace=False, size=len(distrust)-len(trust)))
            avg_trust[epoch_idx, s] = np.median(trust.average(method="median").data[selected_channels], axis=0)
            avg_distrust[epoch_idx, s] = np.median(distrust.average(method="median").data[selected_channels], axis=0)
    for epoch_idx in range(len(EPOCH_TYPES)):
        for sample in range(avg_trust.shape[-1]):
            wilcox_curr[epoch_idx, sample] = ss.wilcoxon(avg_trust[epoch_idx, :, sample], 
                                                         avg_distrust[epoch_idx, :, sample])[1]

    for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
        if "feedback" not in epoch_label:
            continue
        fig, ax = plt.subplots(figsize=(4, 3))
        ep_start, ep_end = EPOCH_TYPES[epoch_label]
        abscissas = np.linspace(ep_start, ep_end, avg_trust.shape[-1])
        ax.axhline(0, color='gray', ls=':', lw=1)
        plt.grid(b=False)
        plt.ylim((-4, 4.25))
        plt.yticks(range(-4, 5, 4), fontsize=18)
        if epoch_label == 'stimulus':
            plt.xticks(range(ep_start+100, ep_end+1, 500), range(ep_start+100+500, ep_end+1+500, 500), fontsize=18)
        else:
            plt.xticks(range(ep_start+100, ep_end+1, 500), range(ep_start+100, ep_end+1, 500), fontsize=18)
        plt.xlim(ep_start+100, ep_end-500)
        p1 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_trust[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Trust", color=TRUST_COLOR, lw=2)
        p2 = ax.plot(abscissas[14:-50:subsample], 1e6*np.median(avg_distrust[epoch_idx], axis=0)[14:-50:subsample], "-", ms=5, label="Distrust", color=DISTRUST_COLOR, lw=2)
        where_is_significant = np.where(wilcox_curr[epoch_idx][14:-50:subsample] < 0.05)[0]
        plt.scatter(abscissas[14:-50:subsample][where_is_significant], [4]*len(where_is_significant), s=40, marker=(5, 2), c='k', lw=0.75)
        plt.legend(loc="lower center", ncol=2, fontsize=14)
        plt.savefig("%strust_vs_distrust/%s_locked/grand_median_%s.pdf" % (GROUP_RESULT_FOLDER, epoch_label, channels))

### Plot scalp maps

In [ ]:
plt.ioff()
for epoch_idx, epoch_label in enumerate(EPOCH_TYPES):
    ep_start, ep_end = EPOCH_TYPES[epoch_label]
    for time in range(ep_start, ep_end, 10):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2.5))
        abscissas = np.linspace(ep_start, ep_end, subject_averages_trust.shape[-1]+1)
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_averages_trust[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        im, cn = mne.viz.plot_topomap(1e6*np.median(subject_averages_distrust[epoch_idx], axis=0)[:, np.where(abscissas == time)[0][0]],
                                      pos=epochs.info, axes=ax2, sensors=False, contours=0, vmin=-10, vmax=10,
                                      show=False, sphere=-1)
        ax2.invert_xaxis()
        ax2.invert_yaxis()
        plt.colorbar(im, ax=[ax1, ax2], ticks=[-10, 0, 10])
        ax1.set_title("Trust")
        ax2.set_title("Distrust")
        plt.savefig("%strust_vs_distrust/%s_locked/scalp_maps_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        fig, (ax1) = plt.subplots(1, 1, figsize=(2.8, 2))
        im, cn = mne.viz.plot_topomap(wilcox_trust[epoch_idx, :, np.where(abscissas == time)[0][0]], cmap='RdBu', vmin=0.0, vmax=0.1,
                                      pos=epochs.info, axes=ax1, sensors=False, contours=0, show=False, sphere=-1,
                                      extrapolate="box", res=200)
        ax1.invert_xaxis()
        ax1.invert_yaxis()
        plt.colorbar(im, ax=[ax1], ticks=[0, 0.05, 0.1])
        plt.savefig("%strust_vs_distrust/%s_locked/scalp_maps_wilcoxon_%dms.pdf" % (GROUP_RESULT_FOLDER, epoch_label, time))
        plt.clf()
        plt.close('all')
plt.ion()

### Frequency

In [ ]:
powers_trust = np.zeros((len(SUBJECTS), 128, 29, 210))
powers_distrust = np.zeros((len(SUBJECTS), 128, 29, 210))
np.random.seed(2)
for s, subject in enumerate(SUBJECTS):
    print("Subject", subject)
    # Load epochs
    trust = mne.read_epochs("%ss%d/s%d_feedback_trust-epo.fif" % (RESULT_FOLDER, subject, subject), 
                              preload=True, verbose=False)
    distrust = mne.read_epochs("%ss%d/s%d_feedback_distrust-epo.fif" % (RESULT_FOLDER, subject, subject), 
                                preload=True, verbose=False)
    # Remove Cz
    trust = trust.drop_channels(["Cz"])
    distrust = distrust.drop_channels(["Cz"])
    # Equalize number of epochs by picking random ones
    if len(trust) > len(distrust):
        trust.drop(np.random.choice(range(len(trust)), replace=False, size=len(trust)-len(distrust)))
    elif len(trust) < len(distrust):
        distrust.drop(np.random.choice(range(len(distrust)), replace=False, size=len(distrust)-len(trust)))
    # Compute PSD
    freqs = np.arange(1., 30., 1.)
    power_trust = mne.time_frequency.tfr_multitaper(trust, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                      return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_trust[s] = power_trust.data
    freqs = power_trust.freqs
    times = power_trust.times
    infos = power_trust.info
    nave = power_trust.nave
    power_distrust = mne.time_frequency.tfr_multitaper(distrust, freqs=freqs, n_cycles=freqs / 4., use_fft=True,
                                                        return_itc=False, n_jobs=8, time_bandwidth=4.0, verbose=False)
    powers_distrust[s] = power_distrust.data

In [ ]:
pvals = np.zeros((len(LOCATIONS), 29, 210))
for f in range(29):
    for t in range(210):
        for l, location in enumerate(LOCATIONS):
            pvals[l, f, t] = ss.wilcoxon(np.median(powers_trust[:, np.array(eval(location))-1, f, t], axis=1),
                                         np.median(powers_distrust[:, np.array(eval(location))-1, f, t], axis=1))[1]

In [ ]:
for l, location in enumerate(LOCATIONS):
    print(location)
    plt.figure(figsize=(8, 8))
    plt.imshow(pvals[l], cmap="RdBu", vmin=0.0, vmax=0.1, interpolation=None, alpha=1, resample=False)
    plt.xticks(range(10, 211, 25), range(0, 2001, 250), fontsize=18)
    plt.xlim(10, 160)
    plt.yticks(range(0, 41, 10), fontsize=18)
    plt.grid(b=False)
    plt.savefig("%s/trust_vs_distrust/psd_wilcoxon_%s.png" % (GROUP_RESULT_FOLDER, location.lower()))

In [ ]:
bands = [['delta', 1, 4], ['theta', 4, 8], ['alpha', 8, 13], ['beta', 13, 30]]
means_per_subject = np.zeros((len(SUBJECTS), len(bands), len(LOCATIONS), 2))
time_from = 0.1
time_to = 0.45
for l, location in enumerate(LOCATIONS):
    for b, band in enumerate(bands):
        for s in range(len(SUBJECTS)):
            subj_trust = np.median(powers_trust[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 1] = 100*np.median(np.sum(subj_trust[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_trust, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
            subj_distrust = np.median(powers_distrust[s, np.array(eval(location))-1], axis=0)
            means_per_subject[s, b, l, 0] = 100*np.median(np.sum(subj_distrust[np.logical_and(freqs >= band[1], freqs <= band[2])], axis=0)[np.logical_and(times >= time_from, times <= time_to)]) / np.median(np.sum(subj_distrust, axis=0)[np.logical_and(times >= time_from, times <= time_to)])
        p = pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1])['p-val'].values[0]
        df = pd.DataFrame()
        df["power"] = means_per_subject[:, b, l].flatten()
        df["condition"] = ["Distrust", "Trust"]*len(SUBJECTS)
        plt.figure(figsize=(2, 3))
        sns.boxplot(data=df, x='condition', y='power', ax=plt.gca(), notch=False, width=0.9, linewidth=1.5,
                    palette=[DISTRUST_COLOR, TRUST_COLOR])
        sns.swarmplot(data=df, x='condition', y='power', ax=plt.gca(), color='0.2')
        plt.xlabel("")
        if band[0] == "alpha":
            plt.yticks([15, 25, 35], fontsize=18)
            plt.ylim(15, 35)
        elif band[0] == "beta":
            plt.yticks([0, 25, 50], fontsize=18)
            plt.ylim(0, 50)
        elif band[0] == "delta":
            plt.yticks([10, 35, 60], fontsize=18)
            plt.ylim(10, 60)
        elif band[0] == "theta":
            plt.yticks([20, 35, 50], fontsize=18)
            plt.ylim(20, 50)
        plt.xticks([])
        plt.ylabel("")
        plt.tight_layout()
        plt.savefig("%s/trust_vs_distrust/eeg_%s_%s.png" % (GROUP_RESULT_FOLDER, band[0], location.lower()))
        if p > 0.0125:
            plt.clf()
            plt.close()
        else:
            print(location, band)
            print(pg.wilcoxon(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            print(pg.compute_effsize(means_per_subject[:, b, l, 0], means_per_subject[:, b, l, 1]))
            plt.show()